In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/non-fulfillment/sample_submission.csv
/kaggle/input/non-fulfillment/train.csv
/kaggle/input/non-fulfillment/test.csv


In [2]:
!pip install -q pytorch-tabnet tab-transformer-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 1.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

# PyTorch 관련 라이브러리
import torch
import torch.nn as nn
import torch.optim as optim

# TabNet (pytorch-tabnet)
from pytorch_tabnet.tab_model import TabNetClassifier

# TabTransformer (tab-transformer-pytorch)
from tab_transformer_pytorch import TabTransformer

In [4]:
# # 데이터 로드 및 전처리
# train_data = pd.read_csv('/kaggle/input/non-fulfillment/train.csv')
# test_data = pd.read_csv('/kaggle/input/non-fulfillment/test.csv')

# # UID 컬럼 제거
# train_data.drop(columns=['UID'], inplace=True)
# test_data.drop(columns=['UID'], inplace=True)

In [5]:
# 1. 데이터 로드 및 UID 제거
train_data = pd.read_csv('/kaggle/input/non-fulfillment/train.csv')
test_data = pd.read_csv('/kaggle/input/non-fulfillment/test.csv')
train_data.drop(columns=['UID'], inplace=True)
test_data.drop(columns=['UID'], inplace=True)


In [6]:
train_data.head()

,주거 형태,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,대출 목적,대출 상환 기간,현재 대출 잔액,현재 미상환 신용액,월 상환 부채액,신용 점수,채무 불이행 여부
0,자가,1941337.5,10년 이상,0.0,9,13.4,400597.5,0,24,1,부채 통합,단기 상환,390903.0,225457.5,8806.5,767,0
1,월세,1979505.0,10년 이상,0.0,5,15.1,360679.5,0,11,0,부채 통합,단기 상환,1002184.5,64749.0,24961.5,767,0
2,월세,1356381.0,4년,0.0,12,18.8,491770.5,1,74,3,부채 통합,단기 상환,227775.0,487644.0,12069.0,800,1
3,월세,1049017.5,6년,0.0,15,14.8,411546.0,1,22,1,부채 통합,단기 상환,251383.5,413211.0,31749.0,796,1
4,월세,4320217.5,2년,0.0,11,26.1,895288.5,0,32,0,부채 통합,장기 상환,1163176.5,78991.5,5862.0,751,0


In [7]:
test_data.head()

,주거 형태,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,대출 목적,대출 상환 기간,현재 대출 잔액,현재 미상환 신용액,월 상환 부채액,신용 점수
0,월세,1560090.0,10년 이상,0.0,13,12.0,495561.0,0,18,0,기타,단기 상환,376332.0,133522.5,29641.5,736
1,주택 담보 대출 (거주 중),2102616.0,2년,0.0,9,29.0,580833.0,0,40,0,부채 통합,장기 상환,830379.0,302983.5,20151.0,718
2,주택 담보 대출 (거주 중),2477989.5,10년 이상,0.0,11,26.5,995841.0,0,44,0,부채 통합,장기 상환,877635.0,379278.0,13113.0,722
3,주택 담보 대출 (거주 중),1571091.0,6년,0.0,7,34.4,601656.0,0,45,0,부채 통합,단기 상환,487278.0,275395.5,11679.0,762
4,주택 담보 대출 (거주 중),2290260.0,10년 이상,0.0,19,25.0,1954623.0,0,14,0,부채 통합,단기 상환,397782.0,742767.0,42370.5,775


In [8]:

# 2. 범주형 변수 처리 (라벨 인코딩 & 원-핫 인코딩)
# 대상 변수: "주거 형태", "현재 직장 근속 연수", "대출 목적", "대출 상환 기간"
label_enc = LabelEncoder()
train_data["현재 직장 근속 연수"] = label_enc.fit_transform(train_data["현재 직장 근속 연수"])
test_data["현재 직장 근속 연수"] = label_enc.transform(test_data["현재 직장 근속 연수"])

train_data = pd.get_dummies(train_data, columns=["주거 형태", "대출 목적", "대출 상환 기간"], drop_first=True)
test_data = pd.get_dummies(test_data, columns=["주거 형태", "대출 목적", "대출 상환 기간"], drop_first=True)

# 3. 수치형 변수 변환 (로그 변환 및 "연체 없음" 컬럼 추가)
log_columns = ["현재 미상환 신용액", "월 상환 부채액", "현재 대출 잔액"]
for col in log_columns:
    train_data[col] = np.log1p(train_data[col])
    test_data[col] = np.log1p(test_data[col])

# "마지막 연체 이후 경과 개월 수"가 0이면 "연체 없음" 컬럼 생성
train_data["연체 없음"] = (train_data["마지막 연체 이후 경과 개월 수"] == 0).astype(int)
test_data["연체 없음"] = (test_data["마지막 연체 이후 경과 개월 수"] == 0).astype(int)

# 4. 결측값 처리 (SimpleImputer: 중간값으로 대체)
imputer = SimpleImputer(strategy="median")
# train_data의 경우, 타겟("채무 불이행 여부")과 특성을 분리하여 처리한 후 다시 합칩니다.
target = "채무 불이행 여부"
train_features = train_data.drop(columns=[target])
train_target = train_data[target]

train_features_imputed = pd.DataFrame(imputer.fit_transform(train_features), 
                                      columns=train_features.columns)
test_data = pd.DataFrame(imputer.transform(test_data), columns=test_data.columns)

# 5. 데이터 스케일링 (StandardScaler 적용)
scaler = StandardScaler()
train_features_scaled = pd.DataFrame(scaler.fit_transform(train_features_imputed), 
                                       columns=train_features_imputed.columns)
test_data = pd.DataFrame(scaler.transform(test_data), columns=test_data.columns)

# 6. 불균형 데이터 처리 (SMOTE 적용)
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(train_features_scaled, train_target)
# 재조합: SMOTE 처리된 데이터프레임을 train_data로 재정의합니다.
train_data = pd.concat([pd.DataFrame(X_resampled, columns=train_features_scaled.columns), 
                        pd.DataFrame(y_resampled, columns=[target])], axis=1)

In [9]:
train_data.head()

,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,현재 대출 잔액,...,대출 목적_여행 자금,대출 목적_의료비,대출 목적_이사 비용,대출 목적_자동차 구매,대출 목적_주택 개보수,대출 목적_주택 구매,대출 목적_휴가 비용,대출 상환 기간_장기 상환,연체 없음,채무 불이행 여부
0,-0.155206,-0.965030,-0.279027,-0.703173,-0.899120,-0.482924,-0.507403,-0.344278,0.742870,-0.165014,...,-0.132682,-0.126703,-0.020004,-0.097934,-0.31621,-0.051057,-0.024502,-0.658553,-0.063372,0
1,-0.128597,-0.965030,-0.279027,-1.568910,-0.663217,-0.507808,-0.507403,-0.993935,-0.442309,1.426284,...,-0.132682,-0.126703,-0.020004,-0.097934,-0.31621,-0.051057,-0.024502,-0.658553,-0.063372,0
2,-0.563024,0.537992,-0.279027,-0.053870,-0.149779,-0.426087,0.302886,2.154402,3.113228,-1.077898,...,-0.132682,-0.126703,-0.020004,-0.097934,-0.31621,-0.051057,-0.024502,-0.658553,-0.063372,1
3,-0.777310,1.139201,-0.279027,0.595432,-0.704847,-0.476099,0.302886,-0.444225,0.742870,-0.911207,...,-0.132682,-0.126703,-0.020004,-0.097934,-0.31621,-0.051057,-0.024502,-0.658553,-0.063372,1
4,1.503292,-0.063217,-0.279027,-0.270305,0.863219,-0.174536,-0.507403,0.055511,-0.442309,1.678079,...,-0.132682,-0.126703,-0.020004,-0.097934,-0.31621,-0.051057,-0.024502,1.518481,-0.063372,0


In [10]:
test_data.head()

,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,현재 대출 잔액,...,대출 목적_소규모 사업 자금,대출 목적_여행 자금,대출 목적_의료비,대출 목적_이사 비용,대출 목적_자동차 구매,대출 목적_주택 개보수,대출 목적_주택 구매,대출 목적_휴가 비용,대출 상환 기간_장기 상환,연체 없음
0,-0.421003,-0.965030,-0.279027,0.162564,-1.093394,-0.423724,-0.507403,-0.644120,-0.442309,-0.229221,...,-0.051057,-0.132682,-0.126703,-0.020004,-0.097934,-0.31621,-0.051057,-0.024502,-0.658553,-0.063372
1,-0.042767,-0.063217,-0.279027,-0.703173,1.265642,-0.370566,-0.507403,0.455300,-0.442309,1.108430,...,-0.051057,-0.132682,-0.126703,-0.020004,-0.097934,-0.31621,-0.051057,-0.024502,1.518481,-0.063372
2,0.218935,-0.965030,-0.279027,-0.270305,0.918725,-0.111852,-0.507403,0.655194,-0.442309,1.201981,...,-0.051057,-0.132682,-0.126703,-0.020004,-0.097934,-0.31621,-0.051057,-0.024502,1.518481,-0.063372
3,-0.413333,1.139201,-0.279027,-1.136042,2.014983,-0.357585,-0.507403,0.705168,-0.442309,0.207467,...,-0.051057,-0.132682,-0.126703,-0.020004,-0.097934,-0.31621,-0.051057,-0.024502,-0.658553,-0.063372
4,0.088054,-0.965030,-0.279027,1.461169,0.710575,0.485848,-0.507403,-0.844014,-0.442309,-0.135529,...,-0.051057,-0.132682,-0.126703,-0.020004,-0.097934,-0.31621,-0.051057,-0.024502,-0.658553,-0.063372


In [11]:
# 타깃 및 피처 분리
label = '채무 불이행 여부'
X = train_data.drop(columns=[label]).values
y = train_data[label].values.astype(np.float32)
X_test = test_data.values

# train/validation 분할
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# # 피처 스케일링 (두 모델 모두 동일하게 사용)
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_val = scaler.transform(X_val)
# X_full = scaler.fit_transform(X)       # 전체 학습 데이터 (TabTransformer 재학습용)
# X_test = scaler.transform(X_test)

In [13]:
# 2. TabNet 모델 학습 (pytorch-tabnet)
tabnet_clf = TabNetClassifier(
    seed=42,
    verbose=1,
    optimizer_params=dict(lr=2e-2)
)

# TabNet은 numpy array를 입력받습니다.
tabnet_clf.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric=['auc'],  # 문자열 대신 리스트로 전달
    max_epochs=100,
    patience=10,
    batch_size=1024,
    virtual_batch_size=128
)


# validation 예측 (TabNet)
tabnet_val_preds = tabnet_clf.predict_proba(X_val)[:, 1]
tabnet_auc = roc_auc_score(y_val, tabnet_val_preds)
print("TabNet Validation AUC:", tabnet_auc)


epoch 0  | loss: 0.75865 | val_0_auc: 0.62118 |  0:00:01s
epoch 1  | loss: 0.6766  | val_0_auc: 0.64938 |  0:00:01s
epoch 2  | loss: 0.66366 | val_0_auc: 0.65568 |  0:00:02s
epoch 3  | loss: 0.65601 | val_0_auc: 0.67318 |  0:00:02s
epoch 4  | loss: 0.65182 | val_0_auc: 0.67696 |  0:00:03s
epoch 5  | loss: 0.64264 | val_0_auc: 0.68981 |  0:00:03s
epoch 6  | loss: 0.64044 | val_0_auc: 0.69197 |  0:00:04s
epoch 7  | loss: 0.63684 | val_0_auc: 0.69804 |  0:00:04s
epoch 8  | loss: 0.63017 | val_0_auc: 0.70438 |  0:00:05s
epoch 9  | loss: 0.62564 | val_0_auc: 0.70531 |  0:00:05s
epoch 10 | loss: 0.62156 | val_0_auc: 0.71004 |  0:00:05s
epoch 11 | loss: 0.62099 | val_0_auc: 0.71494 |  0:00:06s
epoch 12 | loss: 0.61763 | val_0_auc: 0.72112 |  0:00:06s
epoch 13 | loss: 0.61492 | val_0_auc: 0.71825 |  0:00:07s
epoch 14 | loss: 0.61482 | val_0_auc: 0.71714 |  0:00:07s
epoch 15 | loss: 0.61347 | val_0_auc: 0.72061 |  0:00:08s
epoch 16 | loss: 0.61393 | val_0_auc: 0.72499 |  0:00:08s
epoch 17 | los

In [14]:
# # 3. TabTransformer 모델 구현 및 학습 (PyTorch)
# # TabTransformerClassifier 래퍼 정의
# class TabTransformerClassifier(nn.Module):
#     def __init__(self, num_continuous, num_classes=1, dim=32, depth=6, heads=8, mlp_hidden_mults=(4,), dropout=0.1):
#         super().__init__()
#         # categories가 없는 경우에도, 내부에서 연속형 피처를 적절히 임베딩하도록 TabTransformer 초기화
#         self.tabtransformer = TabTransformer(
#             categories=(), 
#             num_continuous=num_continuous, 
#             dim=dim,
#             depth=depth,
#             heads=heads,
#             attn_dropout=dropout,
#             ff_dropout=dropout,
#             mlp_hidden_mults=mlp_hidden_mults
#         )
#         # 추가적인 차원 변환 레이어: 현재 출력(1차원)을 원하는 임베딩 차원(dim)으로 변환
#         self.proj = nn.Linear(1, dim)
#         self.fc = nn.Linear(dim, num_classes)
        
#     def forward(self, x):
#         # 범주형 피처가 없으므로 빈 텐서 생성
#         x_cat = torch.empty(x.size(0), 0, dtype=torch.long, device=x.device)
#         # TabTransformer는 (batch, 1) 형태의 출력을 반환하는 것으로 보임
#         x = self.tabtransformer(x_cat, x)  # x: (batch, 1)
#         # 선형 변환으로 차원 확장: (batch, 1) -> (batch, dim)
#         x = self.proj(x)
#         return self.fc(x)

# # 학습 함수 정의
# def train_tabtransformer(model, X_train, y_train, X_val, y_val, epochs=50, batch_size=256, lr=1e-3, device='cuda' if torch.cuda.is_available() else 'cpu'):
#     model.to(device)
#     criterion = nn.BCEWithLogitsLoss()  # 로짓 출력이므로
#     optimizer = optim.Adam(model.parameters(), lr=lr)
    
#     # 데이터셋 준비
#     train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train, dtype=torch.float32),
#                                                      torch.tensor(y_train, dtype=torch.float32))
#     val_dataset = torch.utils.data.TensorDataset(torch.tensor(X_val, dtype=torch.float32),
#                                                    torch.tensor(y_val, dtype=torch.float32))
#     train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#     val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
#     best_val_auc = 0
#     best_state = None
#     for epoch in range(epochs):
#         model.train()
#         for xb, yb in train_loader:
#             xb, yb = xb.to(device), yb.to(device)
#             optimizer.zero_grad()
#             outputs = model(xb).squeeze(1)  # (batch,)
#             loss = criterion(outputs, yb)
#             loss.backward()
#             optimizer.step()
        
#         # 검증
#         model.eval()
#         all_preds = []
#         all_targets = []
#         with torch.no_grad():
#             for xb, yb in val_loader:
#                 xb, yb = xb.to(device), yb.to(device)
#                 outputs = model(xb).squeeze(1)
#                 preds = torch.sigmoid(outputs)
#                 all_preds.append(preds.cpu().numpy())
#                 all_targets.append(yb.cpu().numpy())
#         all_preds = np.concatenate(all_preds)
#         all_targets = np.concatenate(all_targets)
#         curr_auc = roc_auc_score(all_targets, all_preds)
#         print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Val AUC: {curr_auc:.4f}")
#         if curr_auc > best_val_auc:
#             best_val_auc = curr_auc
#             best_state = model.state_dict()
#     model.load_state_dict(best_state)
#     return model

# num_features = X_train.shape[1]
# tabtrans_model = TabTransformerClassifier(num_continuous=num_features, dim=32, depth=6, heads=8, mlp_hidden_mults=(4,), dropout=0.1)
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# tabtrans_model = train_tabtransformer(tabtrans_model, X_train, y_train, X_val, y_val, epochs=50, batch_size=256, lr=1e-3, device=device)

# # validation 예측 (TabTransformer)
# tabtrans_model.eval()
# with torch.no_grad():
#     X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
#     outputs = tabtrans_model(X_val_tensor).squeeze(1)
#     tabtrans_val_preds = torch.sigmoid(outputs).cpu().numpy()
# tabtrans_auc = roc_auc_score(y_val, tabtrans_val_preds)
# print("TabTransformer Validation AUC:", tabtrans_auc)

In [15]:
# # 4. 앙상블 예측 (두 모델 예측 결과 단순 평균)
# # 테스트 데이터에 대해 예측
# # TabNet 예측
# tabnet_test_preds = tabnet_clf.predict_proba(X_test)[:, 1]

# # TabTransformer 예측
# with torch.no_grad():
#     X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
#     outputs = tabtrans_model(X_test_tensor).squeeze(1)
#     tabtrans_test_preds = torch.sigmoid(outputs).cpu().numpy()

# # 단순 평균 앙상블
# ensemble_preds = (tabnet_test_preds + tabtrans_test_preds) / 2.0

#tabnet 만 
tabnet_test_preds = tabnet_clf.predict_proba(X_test)[:, 1]
ensemble_preds = tabnet_test_preds

In [16]:
# 5. 제출 파일 생성
submission = pd.read_csv('/kaggle/input/non-fulfillment/sample_submission.csv')
submission['채무 불이행 확률'] = ensemble_preds
submission.to_csv('submission_tabnet_tabtransformer_ensemble.csv', index=False)

print("TabNet과 TabTransformer 앙상블을 이용한 예측 및 제출 파일 생성 완료!")

TabNet과 TabTransformer 앙상블을 이용한 예측 및 제출 파일 생성 완료!
